In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb

args = read_args(api_key='../keys/production_api_scicrunch_key.txt', db_url='../keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.db_url)

In [4]:
ex = sql.get_existing_ids()

In [5]:
terms = sql.get_terms()

In [6]:
ex

,id,tid,curie,iri,preferred,ilx,type,label,definition
0,15033,15033,ILX:0115029,http://uri.interlex.org/base/ilx_0115029,1,ilx_0115029,cde,Preinjury Surgical Procedures,Surgical procedures prior to injury
1,15038,15038,ILX:0115034,http://uri.interlex.org/base/ilx_0115034,1,ilx_0115034,cde,Preinjury Conditions,List of preinjury conditions that may have bee...
2,15039,15039,ILX:0115035,http://uri.interlex.org/base/ilx_0115035,1,ilx_0115035,cde,Pre-Injury Subject Housing,Individual or group housing prior to injury.
3,15040,15040,ILX:0115036,http://uri.interlex.org/base/ilx_0115036,1,ilx_0115036,cde,Injury Group Assignment,Injury goup assignment.
4,15041,15041,ILX:0115037,http://uri.interlex.org/base/ilx_0115037,1,ilx_0115037,cde,Investigator,Investigator for a study.
5,12801,12801,ILX:0112800,http://uri.interlex.org/base/ilx_0112800,1,ilx_0112800,cde,UniqueDB code,"Unique database ID for each row of data, indep..."
6,12802,12802,ILX:0112801,http://uri.interlex.org/base/ilx_0112801,1,ilx_0112801,cde,LAB ORG NAME,Name
7,12803,12803,ILX:0112802,http://uri.interlex.org/base/ilx_0112802,1,ilx_0112802,cde,LAB ORG ADDRESS,Address
8,12804,12804,ILX:0112803,http://uri.interlex.org/base/ilx_0112803,1,ilx_0112803,cde,LAB PERSON NAME,Name
9,12805,12805,ILX:0112804,http://uri.interlex.org/base/ilx_0112804,1,ilx_0112804,cde,LAB PERSON AFFILIATION,Affiliation


In [7]:
broken_ex = json.load(open('../dump/broken_exisiting_ids_may_1_2018.json', 'r'))

In [41]:
i = 18
iri = broken_ex[i][1]
if iri == broken_ex[i+1][1] or iri == broken_ex[i-1][1]:
    print('*** DUP ***')
print(broken_ex[i][0])
def curr_ex(iri):
    multi = ex.loc[ex.iri == iri]
    print('==='*20 + ' Total ' + '==='*20)
    display(multi)
    for i, row in multi.iterrows():
        print('\n'+'==='*20 + ' ' + str(i) + ' ' + '==='*20)
        display(ex.loc[ex.tid == row.tid])
        display(terms.loc[terms.id == row.tid])
        grep = !grep -nir $row.curie /home/troy/Desktop/work/NIF-Ontology/
        if grep:
            print(grep)
            
curr_ex(iri)

0105062
============================================================ Total ============================================================


,id,tid,curie,iri,preferred,ilx,type,label,definition
38873,5063,5063,NCBITaxon:6420,http://purl.obolibrary.org/obo/NCBITaxon_6420,1,ilx_0105062,term,Hirudo,
43207,6723,6723,NCBITaxon:6420,http://purl.obolibrary.org/obo/NCBITaxon_6420,1,ilx_0106722,term,Medicinal leech,



============================================================ 38873 ============================================================


,id,tid,curie,iri,preferred,ilx,type,label,definition
38870,5063,5063,NLXWIKI:birnlex_186,http://neurolex.org/wiki/birnlex_186,0,ilx_0105062,term,Hirudo,
38871,5063,5063,BIRNLEX:186,http://uri.neuinfo.org/nif/nifstd/birnlex_186,0,ilx_0105062,term,Hirudo,
38872,5063,5063,ILX:0105062,http://uri.interlex.org/base/ilx_0105062,0,ilx_0105062,term,Hirudo,
38873,5063,5063,NCBITaxon:6420,http://purl.obolibrary.org/obo/NCBITaxon_6420,1,ilx_0105062,term,Hirudo,
38874,5063,5063,GBIF:2308886,http://www.gbif.org/species/2308886,0,ilx_0105062,term,Hirudo,


,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
5062,5063,ilx_0105062,32290,0,32290,0,Hirudo,,,term,4,0,1,1481237111,1516245395



============================================================ 43207 ============================================================


,id,tid,curie,iri,preferred,ilx,type,label,definition
43204,6723,6723,NLXWIKI:birnlex_192,http://neurolex.org/wiki/birnlex_192,0,ilx_0106722,term,Medicinal leech,
43205,6723,6723,BIRNLEX:192,http://uri.neuinfo.org/nif/nifstd/birnlex_192,0,ilx_0106722,term,Medicinal leech,
43206,6723,6723,ILX:0106722,http://uri.interlex.org/base/ilx_0106722,0,ilx_0106722,term,Medicinal leech,
43207,6723,6723,NCBITaxon:6420,http://purl.obolibrary.org/obo/NCBITaxon_6420,1,ilx_0106722,term,Medicinal leech,
43208,6723,6723,GBIF:2308887,http://www.gbif.org/species/2308887,0,ilx_0106722,term,Medicinal leech,


,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
6722,6723,ilx_0106722,32290,0,32290,0,Medicinal leech,,,term,4,0,1,1481240515,1516250738


In [19]:
!python3 /home/troy/Desktop/my_tools/ControlF4Folder.py -t .ttl -f /home/troy/Desktop/work/NIF-Ontology/ -s 'birnlex:656' 

birnlex:656 /home/troy/Desktop/work/NIF-Ontology/ .ttl
| line  ->  10038
| value ->  BIRNLEX:656 a owl:Class ;
| path  ->  /home/troy/Desktop/work/NIF-Ontology/ttl/NIF-Organism.ttl
****************************************************************************************************
| line  ->  10059
| value ->  rdfs:subClassOf BIRNLEX:656 ;
| path  ->  /home/troy/Desktop/work/NIF-Ontology/ttl/NIF-Organism.ttl
****************************************************************************************************
| line  ->  10094
| value ->  rdfs:subClassOf BIRNLEX:656 ;
| path  ->  /home/troy/Desktop/work/NIF-Ontology/ttl/NIF-Organism.ttl
****************************************************************************************************
| line  ->  1024
| value ->  BIRNLEX:656 ilxtr:hasIlxId ILX:0101079 .
| path  ->  /home/troy/Desktop/work/NIF-Ontology/ttl/generated/NIFSTD-ILX-mapping.ttl
****************************************************************************************************

In [11]:

fixes = [
{'':''},# DICOM [0,1] ilx_0100264 ilx_0100265 same dicom but ilx_0100265 was supposed to have 0072 bc it doesnt exist and 0073 does
{'ilx_0109644':'NCBITaxon_8397'},#[2] ilx_0100800 is correct but ilx_0109644 is not and needs to be deleted... NIF-Organism.ttl points to anura from NIF-Eagle-1-Bridge.ttl
{'':''},# DICOM [3] ilx_0100879 is subclassof tiger moths so im pretty sure the child does not own this iri
{'ilx_0106754':'NCBITaxon_9773'},#[4] ref is currect, target is wrong ilx_0106754 is NCBITaxon_9773
{'ilx_0101274':'NCBITaxon_29078'},# [5] Big Brown Bat is wrong
{'':''},# [6] DICOM
{'':''},# [7] DICOM
{'ilx_0101385':'NCBITaxon_7090'}, #[8, 9] Bombyx needs to be corrected
{'ilx_0101619':'NCBITaxon_6753'}, #[10, 11]
{'':''},# [12] DICOM
{'':''},# [13] same as [5] is already handled
{'':''},# [14, 15] DICOM
{'':''},# [16]
{'':''},# [17] ITISTSN:78156
{'ilx_0106722':'NCBITaxon_6421'},# [18]
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
]

SyntaxError: unexpected EOF while parsing (<ipython-input-11-1681254379f1>, line 11)